<a href="https://colab.research.google.com/github/Clamarche/ALAE/blob/master/ELE747_laboratoire2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratoire 2
## ELE747

### PAR 
### Christophe Lamarche
### Mathieu Fournier Desrochers

In [0]:
!python -m pip install -q pillow
!python -m pip install -q scikit-image
!python -m pip install -q scipy

In [0]:
!wget -q https://raw.githubusercontent.com/clamarche/Intersect.Beta/master/images/trees.jpg
!wget -q https://raw.githubusercontent.com/clamarche/Intersect.Beta/master/images/clown.jpeg
!wget -q https://raw.githubusercontent.com/clamarche/Intersect.Beta/master/images/spine.png
!wget -q https://raw.githubusercontent.com/clamarche/Intersect.Beta/master/images/autumn.jpg

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from PIL import Image, ImageDraw
from skimage.measure import profile_line
from skimage import data
from skimage import io
from skimage import transform
from skimage import exposure
from skimage import util

In [0]:
def afficher_grille():
    """
    afficher_grille
    Affiche les quadrillages principales et secondaire dans le plot actuel
    Param:
        {None}
    Return:
        {None}
    """
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    return

def figure(figsize=(11,8)):
    """
    FIGURE  

    Produit une figure de matplotlib  

    @Params :
        {None}
    @Return : 
        {None}
    """
    plt.figure(figsize=figsize,facecolor='w')
    return

#### 1. Filtre passe-haut

##### 1.1 Derivée première de l'intensité
La dérivée première de l'intensité d'une image appelée aussi gradient de l'image est déterminée numériquement en tout pixel sous forme d'un vecteur ayant un module et un argument. Dans les prochaines sections, nous étudions les filtres directionnels de Prewitt et de Sobel.

###### 1.1.1 Filtres de Prewitt
La méthode consiste à convoluer l'image par deux filtres $w_x$ et $w_y$ qui sont respectivement sénsible aux contours verticaux et horizontaux.

$$w_x = \begin{bmatrix}
-1 & 0 & 1\\
-1 & 0 & 1\\
-1 & 0 & 1
\end{bmatrix} \space\space w_y = \begin{bmatrix}
1 & 1 & 1\\
0 & 0 & 0\\
-1 & -1 & -1
\end{bmatrix}$$  

Considérant $f^{'}_{x}=w_x \star f$ et $f^{'}_{y}=w_y \star f$, le module du gradient s'écrit :  
$$D=\sqrt{{f^{'}_{x}}^{2}+{f^{'}_{y}}^{2}}$$  
Une approximation du module du gradient peut être obtenue avec :  
$$D\approx |{f^{'}_{x}}| + |{f^{'}_{y}}|$$  
et l'argument du gradient est donné par : 
$$A=\arctan\big(\frac{f^{'}_{x}}{f^{'}_{y}}\big)$$

###### 1.1.2 Filtres directionnels de Sobel

Les filtres directionnels de Sobel s'appliquent sur les images pour calculer la
dérivée première de l’intensité de la même manière que les filtres de Prewitt. Les
filtres $w_x$ et $w_y$ s'écrivent :  

$$w_x = \begin{bmatrix}
-1 & 0 & 1\\
-2 & 0 & 2\\
-1 & 0 & 1
\end{bmatrix} \space\space w_y = \begin{bmatrix}
1 & 2 & 1\\
0 & 0 & 0\\
-1 & -2 & -1
\end{bmatrix}$$ 

Est-il intéressant d'utiliser l'approximation du module du gradient ? Justifier votre réponse.

Comparer les résultats du filtre de Prewitt avec ceux du filtre de Sobel.

Comment peut-on détecter les contours en utilisant ces filtres (Prewitt et Sobel) ?

##### 1.2 Derivée seconde de l'intensitée

Le Laplacien permet de déterminer la dérivée seconde. Le calcul est basé sur la
convolution de l'image par le filtre $\nabla^{2}$.

$$w_{\nabla^{2}} = \begin{bmatrix}
0 & -1 & 0\\
-1 & 4 & -1\\
0 & -1 & 0
\end{bmatrix}$$

Pourquoi est-il souhaitable d'appliquer un filtre passe-bas avant l'opérateur laplacien ? Justifier votre réponse.

Représenter le profil de l'intensité le long du segment AB sur :
1) l'image initiale spine, 2) le gradient et 3) le Laplacien.

Interpréter dans les trois images l'existence d'un contour. Comment peut-on détecter un contour en utilisant un Laplacien
de Gaussienne ? Enoncer 2 différentes méthodes.

#### 2. Algorithme de Canny
Le détecteur de contours de Canny est un algorithme qui a été conçu pour
détecter les contours en favorisant un faible taux d'erreur, une bonne localisation
en tout offrant une réponse claire.

##### 2.1 Description de l'algorithme

###### 2.1.1 Première étape : Filtre gaussien
Application d’un filtre gaussien :

$$G(x,y,\sigma)=\frac{1}{2\pi\sigma^{2}}e^{-\frac{x^{2}+y^{2}}{2\sigma^{2}}}$$ 

$$G(x,y,\sigma) \star f(x,y)$$

Implémentation de la première étape : filtrage gaussien et calcul du gradient dans deux fonctions ayant les signatures suivantes (n'utilisez pas la fonction fspecial de matlab)
- $I_g = filtrerGaussien(I, \sigma, taille)$
- [M, α] = calculerGradiant($I_g$)

###### 2.1.2 Deuxièmes étapes : Module et angle du gradient 

En utilisant un filtre de dérivé de votre choix (Roberts, Sobel, Prewitt, etc.),
calculez le module (M) et l'argument($\alpha$) du gradient. Notez que l'argument est la
normale du gradient, et que la direction du contour est orthogonale à la normale. Quantifiez ensuite $\alpha$ en 4 orientations : horizontale, $-45^{\circ}$ , verticale et $+45^{\circ}$ notée
respectivement d1, d2, d3 et d4. Fig 3 illustre le résultat attendu avec les angles
correspondent aux normales des contours.

![Figure 3](https://raw.githubusercontent.com/clamarche/Intersect.Beta/master/images/figure3.png)  
Figure 3 - Orientation des contours

Implémentation de la deuxième étape dans une fonction :
- $d_k = determinerDirGrad(\alpha)$

###### 2.1.3 Troisième étape : Suppression des non-maxima

Pour l'ensemble de la matrice de modules (M(x, y))  
— Mettre à zéro les valeurs qui ont au moins un voisin orienté en dk ayant un module de gradient supérieur. Par exemple, considérant un contour horizontal, le gradient sera orienté en d1 et on annule le pixel si son voisin d’en haut M(x, y−1) ou d’en bas M(x, y+1) est supérieur

Implémentation de la troisième étape en une fonction :
- N = supprimerNonMaxima($d_k$,M)

###### 2.1.4 Quatrième étape : Seuillage par hystérésis

- Utilisez deux seuils, $\tau_{L}$ et $\tau_{H}$, pour obtenir les images $T_L$ et $T_H$ respectivement. Comme point de départ pour trouver les $\tau$ optimaux vous pouvez choisir $\tau_{H}$ en observant l’intensité des contour de M avec $\tau_{H} \geq 2\cdot\tau_{L}$. L’image $T_H$ devra contenir des segments de contours désirables (c.-à-d.. dont le gradient est élevé) alors que l’image $T_L$ devra contenir le reste des segments formant les
contours. Parce que le seuil $\tau_{L}$ et plus bas, $T_L$ contiendra également plusieurs
segments ainsi que du bruit indésirable.

- De façon itérative, pour chaque pixel activé dans TH déterminez si un pixel 8-connecté est activé dans TL. Si oui, activez-le dans TH. Répétez jusqu'à ce qu’aucun pixel ne soit ajouté dans TH ou après un nombre limite d’itérations.


Implémentation de la quatrième étape en deux fonctions :
- $[T_L, T_H] = seuillage(N, \tau_l, \tau_H)$
- $I_C = raccord(T_L, T_H)$

#### 3. Applications

Pour les différentes images, utilisez la méthode de votre choix pour répondre au
problème. Justifier votre réponse.

Détecter les contours complets des pièces de monnaie de sorte qu'il serait facile de trouver leurs tailles. On ne veut pas voir apparaître de bruits causés par les gravures internes dans monnaie2.tif.

Détecter le contour des cellules de sorte qu'elles soient faciles à compter. Effectuer le prétraitement à un algorithme qui compte le nombre de cercle et de portion de cercle dans blood1.tif.

Identifier les circuits électriques. On veut vérifier s'il n'y a pas de connexions coupées dans circuit.tif.

Sélectionner les contours des taches blanches de l'image bonemarr.tif.